In [1]:
import numpy
from sklearn.metrics import accuracy_score
import tensorflow as tf
import matplotlib.pyplot as plt

from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import LearningShapelets, \
    grabocka_params_to_shapelet_size_dict
from tslearn.utils import ts_size
import numpy
from sklearn.metrics import accuracy_score

from tslearn.generators import random_walk_blobs
from tslearn.preprocessing import TimeSeriesScalerMinMax, \
    TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier, \
    KNeighborsTimeSeries
import pandas as pd
import numpy as np
import random
from tslearn.datasets import UCR_UEA_datasets
import sktime
from tslearn.preprocessing import TimeSeriesResampler
import random
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_unit_test

In [2]:
df = pd.read_pickle("Train.pkl")

In [3]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [5]:
df.drop(df.loc[(df.Batch == 238)&(df["CuStepNo ValueY"] == 3)].tail(1).index,inplace=True)

In [6]:
df_test = pd.read_csv("Batch_76.csv")
df_test.drop("Unnamed: 0",axis=1,inplace=True)

In [7]:
df_test1 = df_test[:450]

In [8]:
df_test1["CuStepNo ValueY"].unique()[-1]

8

In [9]:
df_test1[df_test1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1]].__len__()

139

In [10]:
df1 = df.loc[df["CuStepNo ValueY"].isin(df_test1["CuStepNo ValueY"].unique())]

In [57]:
df1.loc[(df1["Batch"] == 0)&(df1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1])]

,timestamp,DeviationID ValueY,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,FIC14002_PV_Out ValueY,...,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY,Next_Step,Batch
321,2018-10-10 14:58:40,1,8,141.415466,100,0.059028,1.667824,61.538704,61.544369,1971.671631,...,5.018732,3.960286,15,28.761133,0.200745,0.2,0,0.001743,7.0,0.0
322,2018-10-10 14:58:41,1,8,141.415466,100,0.059028,1.667824,61.538704,61.544369,1971.671631,...,18.917425,3.869575,15,28.654072,0.199648,0.2,0,0.001756,8.0,0.0
323,2018-10-10 14:58:42,1,8,141.401642,100,0.064815,1.517940,60.625454,60.603470,2109.924316,...,18.917425,3.869575,15,28.654072,0.199648,0.2,0,0.001756,8.0,0.0
324,2018-10-10 14:58:43,1,8,141.416473,100,0.065394,1.410301,59.099712,58.950371,2248.942627,...,19.502686,3.867188,15,28.361509,0.195991,0.2,0,0.001755,8.0,0.0
325,2018-10-10 14:58:44,1,8,141.442719,100,0.064815,1.535301,58.702770,58.676903,2129.322021,...,30.939331,3.816262,15,28.361509,0.195991,0.2,0,0.001775,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,2018-10-10 15:00:54,1,8,131.325165,100,0.061343,1.570023,60.190407,60.115421,2076.086426,...,337.313354,7.689815,15,21.977343,0.196344,0.2,0,0.001700,8.0,0.0
456,2018-10-10 15:00:55,1,8,131.159454,100,0.062500,1.625579,60.234848,60.232311,2012.685059,...,338.748291,7.731988,15,21.995602,0.196556,0.2,0,0.001723,8.0,0.0
457,2018-10-10 15:00:56,1,8,131.023224,100,0.059028,1.666667,60.501221,60.507088,1970.676025,...,339.541565,7.763021,15,21.980026,0.196949,0.2,0,0.001735,8.0,0.0
458,2018-10-10 15:00:57,1,8,130.930725,100,0.061921,1.659143,60.634899,60.619713,1978.545044,...,341.065094,7.802807,15,21.868626,0.196277,0.2,0,0.001728,8.0,0.0


In [11]:
df1 = df.loc[df["CuStepNo ValueY"].isin(df_test1["CuStepNo ValueY"].unique())]
for i in df1["Batch"].unique():
    df1.drop(df1.loc[(df1["Batch"] == i)&(df1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1])][df_test1[df_test1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1]].__len__():].index,inplace=True)

C:\Users\paulh\AppData\Local\Temp\ipykernel_13704\1268564111.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop(df1.loc[(df1["Batch"] == i)&(df1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1])][df_test1[df_test1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1]].__len__():].index,inplace=True)


In [12]:
randomlist1 = random.sample(range(0, 260), 78)

df_test1 = df1.loc[df["Batch"].isin(randomlist1)]
df_train1 = df1.loc[~df["Batch"].isin(randomlist1)]

In [40]:
df = pd.read_pickle("Train.pkl")
df.drop("Unnamed: 0",axis=1,inplace=True)
df_train3 = df.loc[~df["Batch"].isin(randomlist1)]
y_train3 = []
X_train3 = df_train3.drop(["Next_Step","timestamp"],axis=1)

In [13]:
X_train1 = df_train1.drop(["Next_Step","timestamp"],axis=1)
X_test1 = df_test1.drop(["Next_Step","timestamp"],axis=1)
y_test1 = []
y_train1 = []

In [14]:
X_train_step1 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)].drop(["Batch","CuStepNo ValueY","DeviationID ValueY"],axis=1))
        y_train1.append(X_train1.loc[(X_train1["Batch"] == i)]["DeviationID ValueY"].unique()[0])
        X_train_step1.append(num)

In [41]:
X_train_step3 = []
for i in X_train3.Batch.unique():
        num = numpy.array(X_train3.loc[(X_train3["Batch"] == i)].drop(["Batch","CuStepNo ValueY","DeviationID ValueY"],axis=1))
        y_train3.append(X_train3.loc[(X_train3["Batch"] == i)]["DeviationID ValueY"].unique()[0])
        X_train_step3.append(num)

In [15]:
X_test_step1 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)].drop(["Batch","CuStepNo ValueY","DeviationID ValueY"],axis=1))
        y_test1.append(X_test1.loc[(X_test1["Batch"] == i)]["DeviationID ValueY"].unique()[0])
        X_test_step1.append(num)

In [99]:
y_test1.__len__()

75

In [95]:
X_train_step1[0][0].__len__()

39

In [42]:
length = []
for i in X_train_step3:
    length.append(i.__len__())   
np.median(length)

1050.0

In [16]:
length = []
for i in X_train_step1:
    length.append(i.__len__())

In [43]:
X_train_s3 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step3)

In [124]:
X_train_s3.shape

(176, 1049, 39)

In [44]:
X_train = X_train_s3.reshape(X_train_s3.shape[0],39,X_train_s3.shape[1])

In [17]:
#median of length
X_train_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step1)
X_test_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step1)

In [18]:
X_train = X_train_s1.reshape(X_train_s1.shape[0],39,X_train_s1.shape[1])
X_test = X_test_s1.reshape(X_test_s1.shape[0],39,X_test_s1.shape[1])

In [108]:
X_train.shape

(176, 39, 459)

In [106]:
np.array(y_train1)

176

In [129]:
X_test.shape

(75, 39, 459)

In [131]:
X_train.shape

(176, 39, 1049)

In [49]:
X_train.shape

(178, 39, 1050)

In [48]:
np.array(y_train1).shape

(178,)

In [50]:
clf = RocketClassifier(num_kernels=300) 
clf.fit(X_train, np.array(y_train3)) 
 

RocketClassifier(num_kernels=300)

In [51]:
y_pred = clf.predict(X_test)

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [39]:
accuracy_score(y_test1, y_pred)

0.9863013698630136

In [56]:
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train", return_X_y=True)
X_test, y_test = load_unit_test(split="test", return_X_y=True) 
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test) 

In [57]:
df = pd.read_pickle("Train.pkl")
randomlist1 = random.sample(range(0, 260), 78)

df_test1 = df.loc[df["Batch"].isin(randomlist1)]
df_train1 = df.loc[~df["Batch"].isin(randomlist1)]
df_steps1 = df_test1.loc[df_test1["CuStepNo ValueY"] != df_test1["Next_Step"]]
df_steps2 = df_train1.loc[df_train1["CuStepNo ValueY"] != df_train1["Next_Step"]]

In [58]:
y_train1 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 1,"DeviationID ValueY"].astype(int)
y_test1 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 1,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)

In [59]:
X_train_step1 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 1)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_train_step1.append(num)

In [60]:
X_test_step1 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 1)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_test_step1.append(num)

In [62]:
length = []
for i in X_train_step1:
    length.append(len(i))

In [91]:
X_train_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step1)
X_test_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step1)

In [54]:
X_train = X_train_s1.reshape(X_train_s1.shape[0],39,X_train_s1.shape[1])
X_test = X_test_s1.reshape(X_test_s1.shape[0],39,X_test_s1.shape[1])

2102      5
3212      1
4263      6
7334      9
8449      9
         ..
260433    9
261541    9
262655    9
263774    9
264886    9
Name: DeviationID ValueY, Length: 174, dtype: int32

In [68]:
X_test_s1.shape[1]

248

In [69]:
X_train = X_train_s1.reshape(X_train_s1.shape[0],39,X_train_s1.shape[1])

In [70]:
X_test = X_test_s1.reshape(X_test_s1.shape[0],39,X_test_s1.shape[1])

In [71]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train, y_train1) 
y_pred = clf.predict(X_test) 

In [45]:
clf.estimator_

SklearnClassifierPipeline(classifier=RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])),
                          transformers=[Rocket(num_kernels=500),
                                        TabularToSeriesAdaptor(transformer=StandardScaler(with_mean=False))])

In [72]:
accuracy_score(y_test1, y_pred)

0.7466666666666667

In [73]:
y_pred

array([ 3,  4,  9,  1,  3,  1,  1,  1,  1,  1,  1,  5,  1,  1,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,
        4,  4,  1,  1,  1,  5,  1,  1,  1,  1,  6,  7,  6,  7,  7, 10,  7,
        1,  7,  6,  7, 10,  9,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  6,  6, 10, 10, 10])

In [75]:
y_train2 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 7,"DeviationID ValueY"].astype(int)
y_test2 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 7,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)

In [74]:
X_train_step7 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 7)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_train_step7.append(num)
X_test_step7 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 7)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_test_step7.append(num)

In [76]:
length = []
for i in X_train_step7:
    length.append(len(i))
X_train_s7 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step7)
X_test_s7 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step7)

In [77]:
X_train_7 = X_train_s7.reshape(X_train_s7.shape[0],39,X_train_s7.shape[1])
X_test_7 = X_test_s7.reshape(X_test_s7.shape[0],39,X_test_s7.shape[1])

In [79]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train_7, y_train2) 
y_pred = clf.predict(X_test_7) 
accuracy_score(y_test2, y_pred)

0.88

In [80]:
y_train8 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 8,"DeviationID ValueY"].astype(int)
y_test8 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 8,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_train_step8 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 8)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_train_step8.append(num)
X_test_step8 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 8)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_test_step8.append(num)

In [81]:
length = []
for i in X_train_step8:
    length.append(len(i))
X_train_s8 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step8)
X_test_s8 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step8)

In [83]:
X_train_8 = X_train_s8.reshape(X_train_s8.shape[0],39,X_train_s8.shape[1])
X_test_8 = X_test_s8.reshape(X_test_s8.shape[0],39,X_test_s8.shape[1])

In [84]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train_8, y_train8) 
y_pred = clf.predict(X_test_8) 
accuracy_score(y_test8, y_pred)

0.92

In [85]:
y_train3 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 3,"DeviationID ValueY"].astype(int)
y_test3 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 3,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_train_step3 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 3)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_train_step3.append(num)
X_test_step3 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 3)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_test_step3.append(num)

In [86]:
length = []
for i in X_train_step8:
    length.append(len(i))
X_train_s3 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step3)
X_test_s3 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step3)

In [87]:
X_train_3 = X_train_s3.reshape(X_train_s3.shape[0],39,X_train_s3.shape[1])
X_test_3 = X_test_s3.reshape(X_test_s3.shape[0],39,X_test_s3.shape[1])

In [88]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train_8, y_train8) 
y_pred = clf.predict(X_test_8) 
accuracy_score(y_test8, y_pred)

0.92